In [2]:
from google.cloud import aiplatform

In [27]:
def hyperparameter_tuning_job(
    dataset_train_path:str,
    dataset_validation_path: str,
    project: str,
    display_name: str,
    image_uri: str,
    root_path: str,
    #package_uri: str,
    #python_module: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com"
):
    from google.cloud import aiplatform
    from google.cloud.aiplatform import gapic as aip
  
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    pipeline_client = aip.PipelineServiceClient(client_options=client_options)

    machine_spec = {
        "machineType": "n1-standard-4",
        #"accelerator_type": aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        #"accelerator_count": 1,
    }
    
    worker_pool_spec = {
        "machineSpec": machine_spec,
        "replicaCount": 1,
         "containerSpec":{
             "imageUri": image_uri,
             "args": ['--dataset_train_path={}'.format(dataset_train_path),
                      '--dataset_validation_path={}'.format(dataset_validation_path)]
         }
        #"python_package_spec": {
        #    "executor_image_uri": executor_image_uri,
            #"package_uris": [package_uri],
            #"python_module": python_module,
           # "args": [],
       # },
    }
    
    base_output_directory = {
        "outputUriPrefix": root_path
        }
    
    # study_spec
    metric = {
        "metricId": "accuracy",
        "goal": 'MAXIMIZE',
    }

    parameter_learning_rate = {
            "parameterId": "learning_rate",
            "doubleValueSpec": {"minValue": 0.001, "maxValue": 0.004},
            "scaleType": 'UNIT_LINEAR_SCALE',
    }
    parameter_iterations = {
        "parameterId": "iteration",
        "integerValueSpec": {"minValue": 70, "maxValue": 75},
        "scaleType": 'UNIT_LINEAR_SCALE',
    }

    # hyperparameter_tuning_job
    training_task_inputs = {
        "maxTrialCount": 4,
        "parallelTrialCount": 2,
        "maxFailedTrialCount": 2,
        "studySpec": {
            "metrics": [metric],
            "parameters": [parameter_learning_rate,parameter_iterations],
            "algorithm": 'RANDOM_SEARCH',
        },
        "trialJobSpec": {"workerPoolSpecs": [worker_pool_spec], "baseOutputDirectory": base_output_directory}
    }
   
    parent = f"projects/{project}/locations/{location}"
    training_task_definition="gs://google-cloud-aiplatform/schema/trainingjob/definition/hyperparameter_tuning_task_1.0.0.yaml"
    
    model_to_upload = aip.Model(
        display_name=display_name,
        container_spec=aip.ModelContainerSpec(
            image_uri=image_uri
        ),
        labels={
            "version": "v1"
        }
    )
    
    
    training_pipeline = aip.TrainingPipeline(
            display_name=display_name,
            training_task_definition=training_task_definition,
            training_task_inputs=training_task_inputs,
            model_to_upload=model_to_upload,
    )

    request =  aip.CreateTrainingPipelineRequest(parent=parent, training_pipeline=training_pipeline)
    running_pipeline = pipeline_client.create_training_pipeline(request)
   
    return  running_pipeline

In [28]:
project_id = 'rugged-shell-320613'
lab_name= 'lab-10'
custom_image= 'gcr.io/rugged-shell-320613/hp-tuning:v5'
location= "us-central1"
api_endpoint= "us-central1-aiplatform.googleapis.com"
dataset_train_path= 'gs://hp-tuning-file/train.csv'
dataset_validation_path = 'gs://hp-tuning-file/validation.csv'
root_path = 'gs://hp-tuning-file/'

In [29]:
job_id = hyperparameter_tuning_job(project=project_id,display_name=lab_name,root_path=root_path,
                                   image_uri=custom_image,dataset_train_path=dataset_train_path,dataset_validation_path=dataset_validation_path)
job_id

name: "projects/383073583641/locations/us-central1/trainingPipelines/4799794865753817088"
display_name: "lab-10"
training_task_definition: "gs://google-cloud-aiplatform/schema/trainingjob/definition/hyperparameter_tuning_task_1.0.0.yaml"
training_task_inputs {
  struct_value {
    fields {
      key: "maxFailedTrialCount"
      value {
        number_value: 2.0
      }
    }
    fields {
      key: "maxTrialCount"
      value {
        number_value: 4.0
      }
    }
    fields {
      key: "parallelTrialCount"
      value {
        number_value: 2.0
      }
    }
    fields {
      key: "studySpec"
      value {
        struct_value {
          fields {
            key: "algorithm"
            value {
              string_value: "RANDOM_SEARCH"
            }
          }
          fields {
            key: "metrics"
            value {
              list_value {
                values {
                  struct_value {
                    fields {
                      key: "goal"
    